In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn import decomposition
from sklearn import manifold
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
from matplotlib import offsetbox
from time import time
from datetime import datetime

In [2]:
df = pd.read_csv("SP500_daily.csv")
pd.set_option('display.expand_frame_repr', False)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (5,6,9,19,48,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#dates = df['date']
new_dates = []
for y in df['date']:
    s = str(y)
    s_date = datetime.strptime(s, '%Y%m%d').strftime('%m/%d/%Y')
    new_dates.append(s_date)

In [5]:
df["new_date"] = new_dates

In [6]:
df.head()

,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,COMNAM,SHRCLS,...,CFACSHR,OPENPRC,NUMTRD,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn,new_date
0,10104,20000103,NaN,11,3,7370,68389X10,ORCL,ORACLE CORP,NaN,...,4.0,124.62500,38710.0,0.054099,-0.006810,-0.006817,0.002878,0.002860,-0.009549,01/03/2000
1,10104,20000104,NaN,11,3,7370,68389X10,ORCL,ORACLE CORP,NaN,...,4.0,115.50000,50279.0,-0.08836,-0.039656,-0.039683,-0.017465,-0.017486,-0.038345,01/04/2000
2,10104,20000105,NaN,11,3,7370,68389X10,ORCL,ORACLE CORP,NaN,...,4.0,101.62500,60672.0,-0.052815,-0.000930,-0.001004,0.007821,0.007743,0.001922,01/05/2000
3,10104,20000106,NaN,11,3,7370,68389X10,ORCL,ORACLE CORP,NaN,...,4.0,100.15625,45177.0,-0.058824,-0.007384,-0.007540,0.004504,0.004453,0.000956,01/06/2000
4,10104,20000107,NaN,11,3,7370,68389X10,ORCL,ORACLE CORP,NaN,...,4.0,95.00000,37044.0,0.076823,0.032519,0.032516,0.017008,0.016991,0.027090,01/07/2000


In [20]:
df1 = df['TICKER'].unique()
df1

array(['ORCL', 'MSFT', 'TROW', 'HON', 'T', 'ADM', 'FRC', 'FISV', 'CF',
       'PHBK', 'BKNG', 'BNK', 'CERN', 'BFO', 'MEC', 'KO', 'CDN', 'CDNS',
       'ED', 'CELG', 'XRAY', 'FAST', 'DTE', 'DD', 'ETN', 'SIVB', 'XOM',
       'MGG', 'MGM', 'MXIM', nan, 'WMI', 'WM', 'GD', 'GE', 'LH', 'PBCT',
       'GM', 'CHTR', 'AIZ', 'LYB', 'IR', 'FLT', 'IBM', 'NLSN', 'KMI',
       'IIN', 'ITT', 'HCA', 'HII', 'KSU', 'MPC', 'FBHS', 'XYL', 'MAY',
       'DLPH', 'APTV', 'TRIP', 'PSX', 'FB', 'PNR', 'FANG', 'ABBV', 'NCLH',
       'ZTS', 'PEP', 'MO', 'Q', 'IQV', 'P', 'COP', 'PVH', 'CDW', 'NWSA',
       'NWS', 'COTY', 'AMGN', 'LIN', 'SLB', 'TWTR', 'ALLE', 'HLT',
       'SNSTA', 'CHV', 'CVX', 'GOOG', 'AAPL', 'AMAT', 'MRKT', 'INFO',
       'ANET', 'SYF', 'CFG', 'KEYS', 'MRO', 'VMC', 'CBS', 'KHC', 'PYPL',
       'TXN', 'HPE', 'EIX', 'UA', 'FTV', 'G', 'LW', 'FTI', 'HSY', 'KR',
       'CVS', 'GIS', 'MHP', 'MHFI', 'SPGI', 'KMB', 'UTX', 'PG', 'SO',
       'CAT', 'CL', 'FMC', 'DE', 'BMY', 'WAG', 'WBA', 'BA', 'UAL', 'HI